# Statistical hypotheses in R. Example 9
## Multiple comparisons test. Numeric data
**Task.** Let’s consider an experiment to compare the effects of three different food regimes on lipoprotein levels in human infants. A group of 10 newborn infants was fed human milk (HM), another group of 10 received milk formula (MF) and a third group received nucleotide supplemented milk formula (NMF). After four weeks, their levels of high density lipoprotein (HDL)
were measured and are given in a table below:

HM|MF|NMF
-|-|-
56|40|71
63|48|57
45|60|64
41|38|44
71|28|73
60|44|50
78|66|79
50|22|67
68|45|84
62|54|61


### 1.	Define the type of the variable
Because the variables are the physical measure (concentration) we're dealing with the numeric data.

**Get data in R**

Before processing the data from the task, one has to take into account that R expects the data for the multiple comparisons to be in a specific format. Each row of an input table has to have name of the group and value of HDL. It's because the multiple comparison methods and corresponding R functions treat multiple group data as a combination of nominal and numeric data. In our case it's one nominal variable (factor in R terms) group GRP with 3 levels: HM, MF, NMF and one numeric variable HDL.

One has to transform the original table in some external software. Any spreadsheet engine of your choice will do (MS Excel, LibreOffice Calc, Google Sheets). The final form of ready to process data is presented in a table below:

GRP|HDL
-|-
HM|56
HM|63
HM|45
HM|41
HM|71
HM|60
HM|78
HM|50
HM|68
HM|62
MF|40
MF|48
MF|60
MF|38
MF|28
MF|44
MF|66
MF|22
MF|45
MF|54
NMF|71
NMF|57
NMF|64
NMF|44
NMF|73
NMF|50
NMF|79
NMF|67
NMF|84
NMF|61


In [1]:
Input = (
    "GRP,HDL
HM,56
HM,63
HM,45
HM,41
HM,71
HM,60
HM,78
HM,50
HM,68
HM,62
MF,40
MF,48
MF,60
MF,38
MF,28
MF,44
MF,66
MF,22
MF,45
MF,54
NMF,71
NMF,57
NMF,64
NMF,44
NMF,73
NMF,50
NMF,79
NMF,67
NMF,84
NMF,61"
)
Data = read.csv(textConnection(Input), header = TRUE, 
                strip.white = TRUE,                # Remove the whitespace characters from the input strings
                stringsAsFactors = TRUE)           # Convert the strings into nominal data
str(Data)

'data.frame':	30 obs. of  2 variables:
 $ GRP: Factor w/ 3 levels "HM","MF","NMF": 1 1 1 1 1 1 1 1 1 1 ...
 $ HDL: int  56 63 45 41 71 60 78 50 68 62 ...


As one can see the imported data frame `Data` holds one nominal variable `GRP` with 3 levels "HM", "MF", "NMF" and one numeric data `HDL`. 

### 2. Check if the samples follow normal distribution
In case one wonders how we get only the needed values for the upcoming normal distribution check, worry not for R has a very concise way to do so. E.g. to get only the values for the human milk HM group one has to type `Data$HDL[Data$GRP == "HM"]`. This notation tells R to get the values of the HDL variable from the whole dataset if it's corresponding GRP variable equals to factor value of "HM". The double equal sign `==` mean equals and is a logical comparison operator. It tests the values on the left and right sides against each other and returns TRUE if both of them are equal, otherwise it return FALSE.

In [2]:
# Normal distribution check for HM group
shapiro.test(Data$HDL[Data$GRP == "HM"])
# Normal distribution check for MF group
shapiro.test(Data$HDL[Data$GRP == "MF"])
# Normal distribution check for NMF group
shapiro.test(Data$HDL[Data$GRP == "NMF"])


	Shapiro-Wilk normality test

data:  Data$HDL[Data$GRP == "HM"]
W = 0.97975, p-value = 0.9638



	Shapiro-Wilk normality test

data:  Data$HDL[Data$GRP == "MF"]
W = 0.9826, p-value = 0.9775



	Shapiro-Wilk normality test

data:  Data$HDL[Data$GRP == "NMF"]
W = 0.98636, p-value = 0.9901


As one can see for every tested group their `p-value` is greater than the significance level of 0.05 which means all the groups follow the normal distribution. We have to use the parametrical methods.

### 3. Formulate the statistical hypotheses

**Null hypothesis (H0):** levels of high density lipoprotein (HDL) are the same for all milk formulae

**Alternative hypothesis (H1):** levels of high density lipoprotein (HDL) are different for all milk formulae

### 4. Testing the hypotheses
The parametrical method for the multiple comparison test for one nominal and one numeric data like in our case is a *one-way ANOVA*. This method is implemented in R as `oneway.test` function. It accepts at least one argument which R names formula and in essence is a short form of representing the relation between the tested variables. The independent variable is specified on the right side of the `~` sign and the dependent variable is provided on the left side. In *one-way ANOVA* the independent variable is always a nominal variable, so the format of the formula argument is `NumericVar ~ NominalVar`.

Second argument `var.equal = TRUE` serves the purpose of initiating the variance comparison which can improve the precision of the `p-value`

In [3]:
oneway.test(Data$HDL ~ Data$GRP, var.equal = TRUE)


	One-way analysis of means

data:  Data$HDL and Data$GRP
F = 7.0909, num df = 2, denom df = 27, p-value = 0.003349


The result of the test shows that we have to reject the null hypothesis (`p-value` < 0.05) and accept the alternative hypothesis.

It means that the levels of high density lipoprotein (HDL) are different for all milk formulae.

We can try to figure out which formula behaves in a different way among the three. To do so we have to perform the pairwise tests for all of the milk formulae. Fortunately we don't have to do it manually and will use the `pairwise.t.test` function.

To get rid of possible errors in the resulting `p-value` we'll have to specify the p-value correction method along with the tested data. We'll use the Bonferroni correction method (for the details one can address this [book](http://www.biostathandbook.com/multiplecomparisons.html)).

In [4]:
pairwise.t.test(Data$HDL, Data$GRP, p.adjust.method = "bonferroni")


	Pairwise comparisons using t tests with pooled SD 

data:  Data$HDL and Data$GRP 

    HM     MF    
MF  0.0401 -     
NMF 0.9855 0.0034

P value adjustment method: bonferroni 

The result of the function is presented as a matrix of `p-values` calculated for each of the group pair. As one can see the substances of HM and NMF get the same levels of HDL in infants while MF gives different levels of HDL. It means that the source of difference shown in the one-way ANOVA test is the administering of milk formula (MF) milk diet.

### Conclusion
Levels of high density lipoprotein (HDL) are different for all milk formulae because of how the milk formula MF diet affects the HDL.
